In [ ]:
!pip install torch

In [ ]:
!pip install -q hydra-core

     |████████████████████████████████| 133kB 3.9MB/s 
     |████████████████████████████████| 112kB 7.1MB/s 
     |████████████████████████████████| 645kB 7.2MB/s 


In [ ]:
!pip install Cython

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.9MB/s 


In [ ]:
!pip install fast-bert

     |████████████████████████████████| 92kB 3.7MB/s 
     |████████████████████████████████| 317kB 22.4MB/s 
     |████████████████████████████████| 3.0MB 37.5MB/s 
     |████████████████████████████████| 778kB 34.0MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 890kB 33.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=e1d7ddd33ffe579210379c53a8db778a8f18e2cdbf1f71f6bba793661a8a3fb2
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c04332f8c0695a613d233f3fe951acc8c7bd82bdade84318a625f440564a1cae
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [ ]:
!pip install apex

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 716kB 8.3MB/s 
     |████████████████████████████████| 256kB 37.2MB/s 
     |████████████████████████████████| 174kB 43.1MB/s 
     |████████████████████████████████| 256kB 40.2MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 143kB 32.6MB/s 
     |████████████████████████████████| 122kB 41.7MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for cryptacular: filename=cryptacular-1.5.5-cp37-abi3-manylinux2010_x86_64.whl size=48025 sha256=439738939b9e2a4949b7d3bfa8c127a8f5ba872ff81648a980cbb7a2d35ed8ed
  Stored in directory: /root/.cache/pip/wheels/3c/79/bc/1eec7120c3ff9b0a2c7ad94d1626abc3388688e2ed7a45878f
Successfully built cryptacular
  Created wheel for apex: filename=apex-0.9.10.dev0-cp37-none-any.whl size=46468 sha256=9ac61607e854e743daf244ed055

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
camembert = torch.hub.load('pytorch/fairseq', 'camembert')

from fastai.imports import *
from transformers import CamembertModel, CamembertTokenizer, CamembertForSequenceClassification, pipeline, AdamW
import pandas as pd

from tqdm import tqdm, trange
import numpy as np

from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from transformers import AdamW

import warnings

In [ ]:
epochs = 5
MAX_LEN = 128
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#model et tokenizer
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
camembert = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=2)
camembert.to(device)

In [ ]:
# verification de l'import de camembert
camembert.eval()

In [ ]:
DATA_PATH = Path('drive/My Drive/trustpilot/data/')
LOG_PATH = Path('drive/My Drive/trustpilot/logs')
MODEL_PATH = Path('drive/My Drive/trustpilot/model/')
LABEL_PATH = Path('drive/My Drive/trustpilot/labels/')

In [ ]:
df = pd.read_csv('drive/My Drive/trustpilot/data/trustpilot_reviews.csv')

In [ ]:
labels = df.columns[1:].to_list()
with open('drive/My Drive/trustpilot/labels/labels.csv', 'w') as f:
  for i in labels:
    print(i)
    f.write(i + '\n')

sentiment


In [ ]:
sentiment_analysis = pipeline("sentiment-analysis",
                              model=camembert,
                              tokenizer=tokenizer)

In [ ]:
# verification du tokenizer
tokenized_sentence = tokenizer.tokenize("J'aime le camembert !")
print(tokenized_sentence)

['▁J', "'", 'aime', '▁le', '▁ca', 'member', 't', '▁!']


In [ ]:
df['sentiment'] = df['sentiment'].astype('int')

In [ ]:
df = df.dropna()
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
df_neg = df[df['sentiment'] == 0]

In [ ]:
df_pos = df[df['sentiment'] == 1].sample(10000)

In [ ]:
df_to_concat = [df_neg, df_pos]

df = pd.concat([df_neg, df_pos])
df = df.sample(frac=1)
df = df.reset_index(drop=True)

print(df)

                                                  review  sentiment
0      première commande. coup de téléphone préalable...          1
1      large choix de produits. qualité et service au...          1
2      commande livrée en quelques jours.  choix de p...          1
3              produit conforme et livre dans les délais          1
4      parfait pour l’accueil téléphonique,disponible...          1
...                                                  ...        ...
13063     commande traitée rapidement et produit extra..          1
13064  super site, très à l'écoute de ses clients,  e...          1
13065                   livraison rapide. a recommander.          1
13066  très bien produit conforme et livraison dans l...          1
13067  livraison rapide,  produits frais et conformes...          1

[13068 rows x 2 columns]


In [ ]:
# Creates list of texts and labels
text = df['review'].to_list()
labels = df['sentiment'].to_list()
print(labels)

[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 

In [ ]:
#user tokenizer to convert sentences into tokenizer
input_ids  = [tokenizer.encode(sent, add_special_tokens=True, truncation=True, max_length=MAX_LEN) for sent in text]

In [ ]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]  
    attention_masks.append(seq_mask)

In [ ]:
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(input_ids, labels, attention_masks,
                                                            random_state=42, test_size=0.1)

In [ ]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
param_optimizer = list(camembert.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps=10e-8)

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# Store our loss and accuracy for plotting if we want to visualize training evolution per epochs after the training process
train_loss_set = []

In [ ]:
# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):  
    # Tracking variables for training
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
  
    # Train the model
    camembert.train()
    for step, batch in enumerate(train_dataloader):
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = camembert(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # Get loss value
        loss = outputs[0]
        # Add it to train loss list
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
    
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    


    # Tracking variables for validation
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Validation of the model
    camembert.eval()
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to device CPU or GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs =  camembert(b_input_ids,token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss, logits = outputs[:2]
    
        # Move logits and labels to CPU if GPU is used
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.1224664854538708


Epoch:  20%|██        | 1/5 [08:21<33:25, 501.47s/it]

Validation Accuracy: 0.9771341463414634
Train loss: 0.058668663716399234


Epoch:  40%|████      | 2/5 [16:43<25:04, 501.62s/it]

...

Validation Accuracy: 0.9740853658536586
Train loss: 0.03160314161963786


Epoch:  80%|████████  | 4/5 [33:27<08:21, 501.81s/it]

Validation Accuracy: 0.9710365853658537
Train loss: 0.027936134475211198


Epoch: 100%|██████████| 5/5 [41:49<00:00, 501.85s/it]

Validation Accuracy: 0.9740853658536586


In [ ]:
# Test the model on a comment
comments = ["j'aime le kebab", "le temps de livraison etait trop long"]

In [ ]:
# Encode the comments
tokenized_comments_ids = [tokenizer.encode(comment,add_special_tokens=True, truncation=True, max_length=MAX_LEN) for comment in comments]
# Pad the resulted encoded comments
tokenized_comments_ids = pad_sequences(tokenized_comments_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks 
attention_masks = []
for seq in tokenized_comments_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

prediction_inputs = torch.tensor(tokenized_comments_ids)
prediction_masks = torch.tensor(attention_masks)

In [ ]:
# Apply the finetuned model (Camembert)
flat_pred = []
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs =  camembert(prediction_inputs.to(device),token_type_ids=None, attention_mask=prediction_masks.to(device))
    logits = outputs[0]
    logits = logits.detach().cpu().numpy() 
    flat_pred.extend(np.argmax(logits, axis=1).flatten())


In [ ]:
for i in range(len(flat_pred)):
    print('Comment: ', comments[i])
    print('Label', flat_pred[i])

Comment:  j'aime le kebab
Label 1
Comment:  le temps de livraison etait trop long
Label 1


In [ ]:
# sauvegarde du modele
# save_path = "/content/drive/MyDrive/trustpilot/model/camembert_sentiment_anal.pt"
# torch.save(camembert.state_dict(), save_path)

In [ ]:
save_path = "/content/drive/MyDrive/trustpilot/model/camembert_model"
torch.save(camembert, "camembert_model")